In [2]:
from dataclasses import dataclass
import itertools
import logging
import random
import math
import numpy as np
import json
import pickle
import time
import torch
import sys
import yaml
import seaborn as sns
import matplotlib.pyplot as plt
from probe_utils import *

from omegaconf import OmegaConf
from pathlib import Path
from torch import nn, Tensor
from torch.nn import functional as F
from typing import List, Optional, Tuple
import os
# os.chdir("/data/tianyu_guo/birth")
from data import DataArgs, Dataset, iterate_batches, make_dataset
from ihead_full_model import ModelArgs, Transformer, forward_hook, test_value, test_sink

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
torch.cuda.set_device(3)

In [4]:
run_path_server = "/data/tianyu/birth/gens/pre-iclr/dynamics/dormant_copy_k3_bos1"
# run_path_server2="/data/tianyu_guo/birth/gens/special/dormant_copy_2"
model, cfg, x, y, ds = load_model(run_path_local="/Users/guotianyu/GitHub/birth/gens/special/markov", run_path_server=run_path_server, n_layers=3, n_heads=1, bos_num=1, train_steps=10000, delim=0, with_data=True, data_path_local="/Users/guotianyu/GitHub/birth/data", data_path_server="/data/tianyu/birth/data")

In [15]:
hook = forward_hook([], '')
pred, outputs_list = model.modified_forward_with_hook(x, hook)

In [26]:
outputs_list[0]['output'].norm(dim=-1)[0, :10]

tensor([140.7036,  27.3864,  17.4664,  27.2720,  32.0737,  31.4242,  31.6992,
         16.8821,  32.4395,  32.2114], grad_fn=<SliceBackward0>)

## Check the gradients

In [17]:
loss = F.cross_entropy(pred.flatten(0, 1), y.flatten(0, 1))
loss.backward()

In [22]:
# Check the gradients of the model's parameters
gradients = {}
for name, param in model.named_parameters():
    gradients[name] = param.grad

In [23]:
gradients.keys()

dict_keys(['tok_embeddings.weight', 'pos_embeddings.weight', 'layers.0.attention.wq.weight', 'layers.0.attention.wk.weight', 'layers.0.attention.wv.weight', 'layers.0.attention.wo.weight', 'layers.0.ff.w1.weight', 'layers.0.ff.w2.weight', 'layers.0.attention_norm.weight', 'layers.0.attention_norm.bias', 'layers.0.ff_norm.weight', 'layers.0.ff_norm.bias', 'layers.1.attention.wq.weight', 'layers.1.attention.wk.weight', 'layers.1.attention.wv.weight', 'layers.1.attention.wo.weight', 'layers.1.ff.w1.weight', 'layers.1.ff.w2.weight', 'layers.1.attention_norm.weight', 'layers.1.attention_norm.bias', 'layers.1.ff_norm.weight', 'layers.1.ff_norm.bias', 'layers.2.attention.wq.weight', 'layers.2.attention.wk.weight', 'layers.2.attention.wv.weight', 'layers.2.attention.wo.weight', 'layers.2.ff.w1.weight', 'layers.2.ff.w2.weight', 'layers.2.attention_norm.weight', 'layers.2.attention_norm.bias', 'layers.2.ff_norm.weight', 'layers.2.ff_norm.bias', 'norm.weight', 'norm.bias', 'output.weight'])

In [24]:
gradients['layers.1.attention.wq.weight'].shape

torch.Size([256, 256])